In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import logging
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
percent_usage = 1
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
dataset = pd.read_csv('data/RecipeNLG_dataset.csv',index_col=[0])

In [ ]:
dataset = dataset.drop(["ingredients","link","source","NER"],axis=1)

In [ ]:
print(dataset.shape)
count , _ =  dataset.shape
count_after = int(count*percent_usage)
dataset = dataset.iloc[:count_after]
print(dataset.shape)

In [ ]:
stop_words = set(stopwords.words('english'))

def cleanText(texts):
    sentences = []
    for idx in tqdm(range(len(texts))):
        for each_sent in sent_tokenize(str(texts[idx])):
            temp_sent = []
            each_sent = each_sent.lower().replace(".","").replace(",","").replace("'","")
            for each_word in each_sent.split():
                if each_word not in stop_words and len(each_word)>= 3:
                    temp_sent.append(each_word)
            sentences.append(temp_sent)
    return sentences

In [ ]:
from ast import literal_eval
dataset["directions"] = dataset["directions"].apply(literal_eval)

In [ ]:
dataset["directions"] = dataset["directions"].map(cleanText)

In [ ]:
dataset.info()

In [ ]:
model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)
documents = list(dataset["directions"])

In [ ]:
model.build_vocab(documents,progress_per=10000)

In [ ]:
model.train(documents,epochs=10)

In [ ]:
model.save("w2v_model")

In [ ]:
model = Word2Vec.load("w2v_model")

In [ ]:
model.wv.most_similar('grill', topn=10)

In [ ]:
model.wv.key_to_index
